In [ ]:
%matplotlib inline
import datetime as dt
import pandas as pd
from QuantLib import *
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## 市场数据
---------------------

In [ ]:
rs = [0.034625, 0.0342, 0.0343,  0.03445, 0.03525, 0.0365, 0.0366, 0.037725, 0.03835, 0.03835]
tenors = [Period(3, Months), Period(6, Months), Period(9, Months), Period(1, Years), Period(2, Years), 
          Period(3, Years), Period(4, Years), Period(5, Years), Period(7, Years), Period(10, Years)]

In [ ]:
quotes = [QuoteHandle(SimpleQuote(r)) for r in rs]

## 构造收益率曲线
---------------------------

In [ ]:
fixedFreq = Quarterly
fixedCalendar = China(China.IB)
fixedDayCount = Actual365Fixed()
fixedConvention = ModifiedFollowing
floatPayTenor = Period(3, Months)
floatingDayCount = Actual365Fixed()
rule = DateGeneration.Backward

iborIndex = Repo(Period(7, Days))
today = fixedCalendar.adjust(Date.todaysDate())
Settings.instance().evaluationDate = today

In [ ]:
instruments = [SubPeriodsSwapRateHelper(r, t, fixedFreq, fixedCalendar,
                                        fixedDayCount, fixedConvention,
                                        floatPayTenor, iborIndex, floatingDayCount,
                                       rule) for r, t in zip(quotes, tenors)]

In [ ]:
curve = PiecewiseLinearZero(today, instruments, Actual365Fixed())

## 获取收益率曲线数据
-----------------------

In [ ]:
pillars = [today + t for t in tenors]
py_dates = [dt.datetime(d.year(), d.month(), d.dayOfMonth()) for d in pillars]
zero_rates = [curve.zeroRate(d, Actual365Fixed(), Continuous).rate() for d in pillars]

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Chinese Repo-7d/3M Swap Zero Rate curve", fontsize=16)
plt.plot(py_dates, zero_rates)
plt.legend(['zero_rate'])